# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import gmaps.datasets

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:

# Load CSV file generated from WeatherPy Folder
weather = "weather_df.csv"
weather1 = pd.read_csv(weather)
weather_data = weather1.dropna()
del weather_data["Unnamed: 0"]
weather_data.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,pokhara,NP,2022-06-19 22:51:59,28.2333,83.9833,96.0,84.0,73.148,2.71
1,manzanillo,CU,2022-06-19 22:52:00,20.3433,-77.1167,100.0,78.0,76.190,0.14
2,rikitea,PF,2022-06-19 22:52:00,-23.1203,-134.9692,100.0,73.0,72.680,6.05
3,carnarvon,AU,2022-06-19 22:52:01,-24.8667,113.6333,1.0,70.0,68.594,5.59
4,hami,CN,2022-06-19 22:52:02,42.8000,93.4500,100.0,11.0,89.294,2.14


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_data[["Latitude", "Longitude"]]

humidity = weather_data["Humidity (%)"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Filter vacation with zero cloudiness
vacation_no_cloud = weather_data[weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
slow_wind_df = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
vacation = slow_wind_df[slow_wind_df["Humidity (%)"] < 60]
# Set Index
perfect_vacation = vacation.reset_index()
del perfect_vacation["index"]
perfect_vacation.dropna()
perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,karratha,AU,2022-06-19 22:53:40,-20.7377,116.8463,0.0,50.0,76.550,1.76
1,avera,US,2022-06-19 22:54:18,33.1940,-82.5271,0.0,31.0,72.824,6.32
2,russell,US,2022-06-19 22:54:30,32.3502,-85.1999,0.0,53.0,74.282,2.57
3,rakhiv,UA,2022-06-19 22:55:52,48.0526,24.2009,0.0,43.0,79.934,2.46
4,bocaiuva,BR,2022-06-19 22:56:27,-17.1078,-43.8150,0.0,29.0,73.382,5.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:

# Store into variable named hotel_df:
hotel_df = slow_wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df  
    

c:\Users\user\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\Users\user\anaconda3\envs\bootcamp\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Issues with finding a hotel at avera
Issues with finding a hotel at russell


,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
114,glendive,US,2022-06-19 22:53:17,47.1053,-104.7125,0.0,93.0,71.456,5.66,"Holiday Inn Express & Suites Glendive, an IHG ..."
137,bartlesville,US,2022-06-19 22:52:47,36.7473,-95.9808,0.0,87.0,78.296,0.89,Hilton Garden Inn Bartlesville
150,karratha,AU,2022-06-19 22:53:40,-20.7377,116.8463,0.0,50.0,76.550,1.76,Aspen Karratha Village
205,avera,US,2022-06-19 22:54:18,33.1940,-82.5271,0.0,31.0,72.824,6.32,NA
223,russell,US,2022-06-19 22:54:30,32.3502,-85.1999,0.0,53.0,74.282,2.57,NA
269,bismarck,US,2022-06-19 22:55:01,46.8083,-100.7837,0.0,65.0,78.926,5.14,Days Inn by Wyndham Bismarck
343,rakhiv,UA,2022-06-19 22:55:52,48.0526,24.2009,0.0,43.0,79.934,2.46,Приватна садиба ''Тушер''
365,manicore,BR,2022-06-19 22:56:07,-5.8092,-61.3003,0.0,83.0,77.216,1.03,Curica Hotel
394,bocaiuva,BR,2022-06-19 22:56:27,-17.1078,-43.8150,0.0,29.0,73.382,5.29,Pontual Hotel
413,saint-pierre,RE,2022-06-19 22:56:40,-21.3393,55.4781,0.0,60.0,75.146,5.66,"Villa Delisle, Hotel & Spa"


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Checking hotel info:
len(hotel_info)

14

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))